In [4]:
import cv2
import numpy as np
import tkinter as tk
from PIL import Image, ImageTk

# HSV Ranges
LOWER_SILVER, UPPER_SILVER = np.array([0, 0, 80]), np.array([180, 60, 255])
LOWER_GOLD, UPPER_GOLD = np.array([10, 80, 80]), np.array([40, 255, 255])

def classify_coin(hsv, silver_mask, gold_mask, x, y, r):
    def count(mask, radius):
        temp = np.zeros(hsv.shape[:2], np.uint8)
        cv2.circle(temp, (x, y), radius, 255, -1)
        return cv2.countNonZero(cv2.bitwise_and(mask, mask, mask=temp))

    full, center = r, int(r * 0.55)
    silver_full, gold_full = count(silver_mask, full), count(gold_mask, full)
    silver_center, gold_center = count(silver_mask, center), count(gold_mask, center)
    silver_ring, gold_ring = silver_full - silver_center, gold_full - gold_center

    # Ratios
    def ratio(a, b): return a / (a + b + 1)
    s_full, g_full = ratio(silver_full, gold_full), ratio(gold_full, silver_full)
    s_center, g_center = ratio(silver_center, gold_center), ratio(gold_center, silver_center)
    s_ring, g_ring = ratio(silver_ring, gold_ring), ratio(gold_ring, silver_ring)

    if s_full > 0.7: return "1 Peso", 1
    if g_full > 0.7: return "5 Peso", 5
    if g_center > 0.5 and s_ring > 0.5: return "10 Peso", 10
    if s_center > 0.5 and g_ring > 0.5: return "20 Peso", 20
    return "Unknown", 0

# Setup camera
cap = cv2.VideoCapture(1)
cap.set(3, 640), cap.set(4, 480)

# Setup GUI (landscape layout)
root = tk.Tk()
root.title("Coin Detector")

panels = [tk.Label(root) for _ in range(4)]
for i, p in enumerate(panels):
    p.grid(row=0, column=i, padx=5, pady=5)  # all in one row

# Labels for results
total_label = tk.Label(root, text="Total: 0 PHP", font=("Arial", 16), fg="red")
coin_label = tk.Label(root, text="Coins: 0", font=("Arial", 16), fg="blue")
total_label.grid(row=1, column=0, columnspan=2, pady=10)
coin_label.grid(row=1, column=2, columnspan=2, pady=10)

def update_frame():
    ret, frame = cap.read()
    if not ret:
        return root.after(10, update_frame)

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    silver_mask, gold_mask = cv2.inRange(hsv, LOWER_SILVER, UPPER_SILVER), cv2.inRange(hsv, LOWER_GOLD, UPPER_GOLD)

    # Circle detection
    gray = cv2.GaussianBlur(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), (9, 9), 2)
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 40, param1=100, param2=25, minRadius=15, maxRadius=80)

    output, total, coin_count = frame.copy(), 0, 0
    if circles is not None:
        for x, y, r in np.uint16(np.around(circles[0])):
            cv2.circle(output, (x, y), r, (0, 255, 0), 2)
            coin, val = classify_coin(hsv, silver_mask, gold_mask, x, y, r)
            total += val
            coin_count += 1  # count every detected coin
            cv2.putText(output, coin, (x - r, y - r - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    cv2.putText(output, f"Total: {total} PHP", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
    cv2.putText(output, f"Coins: {coin_count}", (20, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 3)

    # Display in panels
    imgs = [
        ("Original Image", frame), ("Silver Mask", cv2.cvtColor(silver_mask, cv2.COLOR_GRAY2BGR)),
        ("Gold Mask", cv2.cvtColor(gold_mask, cv2.COLOR_GRAY2BGR)), ("Final Detected Object", output)
    ]
    for panel, (title, img) in zip(panels, imgs):
        im = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)).resize((320, 240))
        panel.imgtk = ImageTk.PhotoImage(im)
        panel.config(image=panel.imgtk, text=title, compound="top")

    total_label.config(text=f"Total: {total} PHP")
    coin_label.config(text=f"Coins: {coin_count}")
    root.after(10, update_frame)

def on_closing():
    cap.release()
    root.destroy()

root.protocol("WM_DELETE_WINDOW", on_closing)
update_frame()
root.mainloop()


C:\Users\Jhon Paul Ojales\AppData\Local\Temp\ipykernel_83532\1627421907.py:70: RuntimeWarning: overflow encountered in scalar subtract
  cv2.putText(output, coin, (x - r, y - r - 10),
